### Extract GRIB Data

In [ ]:
import pandas as pd
import xarray as xr
import os

In [ ]:
def GRIB_to_df(region, filepath):
    """
    Converts a GRIB file to a CSV file. 
    Inputs:
        - filepath: string containing the path to the GRIB data
        - region: string denoting the region of the data
    Returns:
    """

    # Open GRIB file using cfgrib engine
    ds = xr.open_dataset(filepath, engine="cfgrib")

    # Average across latitude and longitude
    regional_means = ds.mean(dim=["latitude", "longitude"])

    # Make DataFrame
    df = regional_means.to_dataframe()

    # Drop unnecessary columns
    df.drop(columns=['valid_time', 'surface', 'step'], inplace=True)

    # Rename columns
    df.rename(columns={
        't2m': f'temp_2m_{region}',
        'u100': f'wind_u_100m_{region}',
        'v100': f'wind_v_100m_{region}'
    }, inplace=True)

    return df


In [ ]:
# Define file path
filepath_dict = {'south':'raw_era5/era5_south.grib', 
             'north':'raw_era5/era5_north.grib',
             'east':'raw_era5/era5_east.grib', 
             'west':'raw_era5/era5_west.grib'}

region_dfs = []

# Convert each GRIB file into a dataframe and save in list
for region in filepath_dict:
    region_df = GRIB_to_df(region, filepath_dict[region])
    region_dfs.append(region_df)

# Merge dataframes into one
df_merged = pd.concat(region_dfs, axis=1)

In [4]:
# Save to CSV
clean_data_path = os.path.join('..', 'clean_data')
output_path = os.path.join(clean_data_path, 'era5_by_region.csv')
df_merged.to_csv(output_path)

print('Sucessfully saved as a CSV!')

Sucessfully saved as a CSV!
